In [ ]:
from atmospheric_explorer.cams_interfaces import InversionOptimisedGreenhouseGas, EAC4Instance
from atmospheric_explorer.shapefile import ShapefilesDownloader
from atmospheric_explorer.utils import get_local_folder
from atmospheric_explorer.units_conversion import convert_units_array
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import xarray as xr
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping
import rioxarray
import plotly.graph_objects as go
import shutil
import numpy as np
import statsmodels.stats.api as sms
import pandas as pd
import cdsapi
from atmospheric_explorer.plotting_apis import eac4_hovmoeller_levels_plot
from atmospheric_explorer.data_transformations import clip_and_concat_countries, shifting_long

In [ ]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'data'))

In [ ]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'shapefiles'))

In [ ]:
#function to move from 0+360 to -180+180 long
def ds_swaplon(ds):
    return ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180)).sortby('longitude')

In [ ]:
sh_down = ShapefilesDownloader(
    resolution='10m',
    instance='countries'
)
sh_down.get_as_dataframe()

In [ ]:
sh = gpd.read_file(sh_down.shapefile_full_path, crs='EPSG:4326')

In [ ]:
manager2 = EAC4Instance(
    'total_column_nitrogen_dioxide',
    'netcdf',
    dates_range = '2022-01-01/2022-10-31',
    time_values = '00:00'
)
manager2.download()

In [ ]:
df2 = xr.open_dataset(manager2.file_full_path)
df2=ds_swaplon(df2)
df2

In [ ]:
df2 = df2.rio.write_crs('EPSG:4326')

In [ ]:
portugal_shapefile = sh[sh['ADMIN'] == 'Portugal']

In [ ]:
#all_touched=True questo parametro include tutti i pixel toccati dal poligono definito, se False include solo i pixel il cui centro è incluso nel poligono
#approvato all_touched=True
df2_clipped = df2.rio.clip(portugal_shapefile.geometry.apply(mapping), sh.crs, drop=True, all_touched=True)

In [ ]:
df2_agg = df2_clipped.mean(dim=['latitude', 'longitude']).resample(time="1MS", restore_coord_dims=True).mean(dim='time')

In [ ]:
df2_agg

In [ ]:
reference_value = df2_agg.mean(dim='time')

In [ ]:
reference_value

In [ ]:
df2_agg

In [ ]:
df2_agg_convert = convert_units_array(df2_agg['tcno2'], 'total_column_nitrogen_dioxide')

In [ ]:
reference_value = df2_agg_convert.mean().values

In [ ]:
df2_agg_anomalies = df2_agg_convert - reference_value
df2_agg_anomalies.attrs = df2_agg_convert.attrs

In [ ]:
df2_agg_convert

In [ ]:
fig = px.line(
    y=df2_agg_anomalies.values,
    x=df2_agg_anomalies.coords['time'],
    markers='o'
)
fig.update_xaxes(title="Month")
fig.update_yaxes(title=df2_agg_anomalies.attrs['units'])
fig.update_layout(
    title={
        "text": "Total columns N02 above Portugal (anomalies)",
        "x": 0.45,
        "y": 0.95,
        "automargin": True,
        "yref": "container",
        "font": {
            "size":19
        }
    }
)

In [ ]:
df2.rio.write_crs("epsg:4326", inplace=True)
#import shapefile and define reference system of shp
#clipping
df2_clip = df2.rio.clip(portugal_shapefile.geometry.apply(mapping), portugal_shapefile.crs, drop=True)
df2_res =(df2_clip.resample(time="1M", restore_coord_dims=True).mean(dim='time'))
#simple plot
df2_res.tcno2[0].plot()